In [1]:
from __future__ import print_function
import shap
import numpy as np
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
#To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
import os

In [2]:
print(tf.version)

<module 'tensorflow._api.v2.version' from 'C:\\Users\\ayoub\\.conda\\envs\\shap\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [3]:
# Proprocessing the test and training set
train_datagen = ImageDataGenerator(rescale = 1./255, #pixelwaarden normaliseren
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32, #hoeveel foto's per keer langs neural netwerk laat passeren
                                                 class_mode = 'binary')
#fit from generator
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/testing',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 3047 images belonging to 2 classes.
Found 1309 images belonging to 2 classes.


In [4]:
# Initialising the CNN
cnn = tf.keras.models.Sequential() # layers gaan op elkaar volgen

# Add convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Add convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

# Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))

# Flattening
cnn.add(tf.keras.layers.Flatten()) # full connection maken (zorgt voor features)

# Full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # sigmoid = input transformeren naar 0-1

# Compile CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

cnn.fit(x = training_set, validation_data = test_set, epochs = 1)

%tensorboard --logdir logs/fit

96/96 [==============================] - 16s 155ms/step - loss: 0.0734 - accuracy: 0.9902 - val_loss: 0.0448 - val_accuracy: 0.9931


In [5]:
# Making a prediction
test_image = image.load_img('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/testing/waldo/12_2_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  print("Tested negative for Waldo :'(")
else:
  print('Tested positive for Waldo! :D')

Tested positive for Waldo! :D


In [6]:
x_train, y_train = next(training_set)
x_test, y_test = next(test_set)
#model.fit(x = training_set, validation_data = test_set, epochs = 1)
score = cnn.evaluate(test_set, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.04479275643825531
Test accuracy: 0.9931245446205139


In [7]:
# select a set of background examples to take an expectation over
background = x_train[np.random.choice(x_train.shape[0], 100, replace=True)]

In [8]:
# explain predictions of the model (named cnn in this case) on three images
#e = shap.DeepExplainer(cnn, background)
e = shap.DeepExplainer((cnn.layers[0].input, cnn.layers[-1].output), background)

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.


In [16]:
from tensorflow.python.keras import backend
graph = backend.get_graph()
print(graph)
#return explainer.model_output.graph

FuncGraph(name=keras_graph, id=2319555082656)


In [9]:
shap_values = e.shap_values(x_test[1:5][1:5])
print(shap_values)

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


[array([[[[-1.26543682e-07,  5.32326574e-08,  1.76521823e-07],
         [-1.27817486e-07, -1.52756954e-08, -9.46584766e-09],
         [-1.05576464e-06, -2.03276244e-07, -1.73813987e-07],
         ...,
         [ 2.16237225e-07, -1.91443263e-07,  2.06228009e-07],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 5.25475308e-08, -1.40699470e-08, -5.91677018e-09],
         [-6.44168665e-08,  3.22849331e-07, -9.34171283e-08],
         [ 1.35796796e-07, -4.69640931e-08,  1.00078836e-07],
         ...,
         [-7.18697265e-07,  7.83967664e-08, -3.91629214e-07],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[-5.50288313e-08, -2.69962129e-07,  5.63896265e-07],
         [-1.48587404e-07, -1.80071777e-06, -7.17856040e-07],
         [ 1.57780899e-08,  9.28486941e-08, -4.37445919e-07],
         ...,
         [-3.77950755e-